In [31]:
import re
import json
import pytz
import string
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from typing import List

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

---

### Dados da B3 (Dados das Empresas Listadas)

**Colunas de interesse**

- `tickers`
- `nome_pregao`
- `setor_economico`


In [61]:
tickers = ['B3SA3', 'EQTL3', 'ITUB4', 'PETR4', 'VALE3']

In [62]:
with open('./results-b3.json', encoding='utf8') as json_file:
    data = json.load(json_file)

df_b3 = pd.DataFrame(data)
df_b3.sample(5)

,razao_social,nome_pregao,segmento,tickers,setor_economico,subsetor,segmento_setorial,url_dados
201,MAHLE-METAL LEVE S.A.,METAL LEVE,NM,[LEVE3],Consumo Cíclico,Automóveis e Motocicletas,Automóveis e Motocicletas,http://bvmf.bmfbovespa.com.br/pt-br/mercados/a...
473,ALFA HOLDINGS S.A.,ALFA HOLDING,NDA,"[RPAD3, RPAD5, RPAD6]",Financeiro,Intermediários Financeiros,Bancos,http://bvmf.bmfbovespa.com.br/pt-br/mercados/a...
358,CONC DO AEROPORTO INTERNACIONAL DE GUARULHOS S.A.,GRUAIRPORT,MB,[],Bens Industriais,Transporte,Serviços de Apoio e Armazenagem,http://bvmf.bmfbovespa.com.br/pt-br/mercados/a...
123,PRODUTORES ENERGET.DE MANSO S.A.- PROMAN,PROMAN,MB,[PRMN3B],Utilidade Pública,Energia Elétrica,Energia Elétrica,http://bvmf.bmfbovespa.com.br/pt-br/mercados/a...
54,TERMINAL GARAGEM MENEZES CORTES S.A.,MENEZES CORT,MB,[MNZC3B],Financeiro,Exploração de Imóveis,Exploração de Imóveis,http://bvmf.bmfbovespa.com.br/pt-br/mercados/a...


In [63]:
df_final = pd.DataFrame()

for index, row in df_b3.iterrows():
    for ticker in row['tickers']:
        if ticker in tickers:
            df_final = df_final.append(row)

In [64]:
df_final.sort_values('razao_social')

,razao_social,nome_pregao,segmento,tickers,setor_economico,subsetor,segmento_setorial,url_dados
450,B3 S.A. - BRASIL. BOLSA. BALCÃO,B3,NM,[B3SA3],Financeiro,Serviços Financeiros Diversos,Serviços Financeiros Diversos,http://bvmf.bmfbovespa.com.br/pt-br/mercados/a...
296,EQUATORIAL ENERGIA S.A.,EQUATORIAL,NM,[EQTL3],Utilidade Pública,Energia Elétrica,Energia Elétrica,http://bvmf.bmfbovespa.com.br/pt-br/mercados/a...
236,ITAU UNIBANCO HOLDING S.A.,ITAUUNIBANCO,N1,"[ITUB3, ITUB4, ITUB99]",Financeiro,Intermediários Financeiros,Bancos,http://bvmf.bmfbovespa.com.br/pt-br/mercados/a...
138,PETROLEO BRASILEIRO S.A. PETROBRAS,PETROBRAS,N2,"[PETR3, PETR4]",Petróleo. Gás e Biocombustíveis,Petróleo. Gás e Biocombustíveis,Exploração. Refino e Distribuição,http://bvmf.bmfbovespa.com.br/pt-br/mercados/a...
31,VALE S.A.,VALE,NM,[VALE3],Materiais Básicos,Mineração,Minerais Metálicos,http://bvmf.bmfbovespa.com.br/pt-br/mercados/a...


In [68]:
nomes_pregao = df_final.nome_pregao.values.tolist()
nomes_pregao

['VALE', 'PETROBRAS', 'ITAUUNIBANCO', 'EQUATORIAL', 'B3']

In [100]:
ticker_name = {
    'B3SA3': 'B3',
    'EQTL3': 'EQUATORIAL',
    'ITUB4': 'ITAUUNIBANCO',
    'PETR4': 'PETROBRAS',
    'VALE3': 'VALE'
}

---

### Dados do Portal de Notícias Suno Research

In [ ]:
with open('../crawlers/suno/results/suno-petr4.json', encoding='utf8') as json_file:
    data = json.load(json_file) 
    
df_suno = pd.DataFrame(data)

In [ ]:
# Convert date to datetime format
df_suno['date'] = pd.to_datetime(df_suno['date'], format='%d/%m/%Y %H:%M')

# Set date column as index
df_suno.set_index('date', inplace=True)

# Order by date
df_suno.sort_index(inplace=True)

In [32]:
df_suno.head()

,topic,title,search_date,url,tags
date,,,,,
2020-07-16 08:27:00,Negócios,Agenda do Dia: JHSF; Tim; Oi; Petrobras; Metal...,2022-05-03 16:22:46,https://www.suno.com.br/noticias/agenda-do-dia...,"[agenda do dia, JHSF (JHSF3), Metalfrio (FRIO3..."
2020-07-16 09:46:00,Negócios,Petrobras (PETR4) conclui venda de campos pelo...,2022-05-03 16:22:45,https://www.suno.com.br/noticias/petrobras-pet...,"[PETR3, Petrobras (PETR4)]"
2020-07-16 17:28:00,Mercado,"Ibovespa hoje cai 1,22% em mau humor com bater...",2022-05-03 16:22:45,https://www.suno.com.br/noticias/ibovespa-fech...,"[Ibovespa, JHSF (JHSF3), Petrobras (PETR4), Va..."
2020-07-16 19:04:00,Negócios,Petrobras (PETR4): exportação de petróleo para...,2022-05-03 16:22:44,https://www.suno.com.br/noticias/petrobras-pet...,"[China, PETR3, Petrobras (PETR4), Petróleo]"
2020-07-17 09:06:00,Negócios,Agenda do Dia: Moura Dubeux; Tenda; Petrobras;...,2022-05-03 16:22:44,https://www.suno.com.br/noticias/agenda-do-dia...,"[agenda do dia, Cia Hering (HGTX3), Moura Dube..."


---

### Rule-Based Method for NER

- `Suno Research`
  - Do nothing (for now) since the news portal already filter by ticker in the crawler
- `Twitter`
  - Store original tweet and mentioned ticker

#### Suno Research

In [34]:
def get_word_list(sentence: str) -> List[str]:
    list_of_words = [[w.upper().translate(str.maketrans('', '', string.punctuation))] for w in sentence.split()]
    return list_of_words

def get_tag_list(tag_list: List[str]) -> List[List[str]]:
    list_of_tags = [re.findall(r"[\w']+|[.,!?;]", x.upper()) for x in tag_list]
    return list_of_tags

In [82]:
tz = pytz.timezone('America/Sao_Paulo')

df_suno_t = pd.DataFrame()

for index, row in df_suno.iterrows():

    title = str(row['title'])
    tags = row['tags']

    # Lista de palavras, sem pontuação, com letras maiúsculas
    list_of_words = get_word_list(title)

    # Lista de tags, sem pontuação, com letras maiúsculas
    list_of_tags = get_tag_list(tags)

    # [TAGS]
    company_names_set = set()

    # Flat tag list(e.g. [['AGENDA', 'DO', 'DIA'], ['JHSF', 'JHSF3']] -> ['AGENDA', 'DO', 'DIA', 'JHSF', 'JHSF3'])
    tags_flat_list = [item.translate(str.maketrans('', '', string.punctuation)) for sublist in list_of_tags for item in sublist]

    # 1. Verifica ocorrência exata ou do nome do pregão na lista de TAGS
    for tag_word in tags_flat_list:
        if (tag_word in tickers):
            company_names_set.add(ticker)
        if (tag_word in nomes_pregao):
            position = list(ticker_name.values()).index(tag_word)
            company_names_set.add(list(ticker_name.keys())[position])

    # Remover pontuação
    word_flat_list = [item.translate(str.maketrans('', '', string.punctuation)) for sublist in list_of_words for item in sublist]
    
    if 'VALE' in word_flat_list:
        word_flat_list.remove('VALE')

    # [TITLE]
    # 2. Verifica ocorrência exata ou do nome do pregão no TITLE
    for title_word in word_flat_list:
        if title_word in nomes_pregao:          
            position = list(ticker_name.values()).index(title_word)
            company_names_set.add(list(ticker_name.keys())[position])

    mentioned_tickers = set(word_flat_list).intersection(tickers)
    mentioned_tickers.update(company_names_set)

    if mentioned_tickers and ('VALE3' in mentioned_tickers):
        print (f'Headline: {str(row.title)}')
        for ticker in list(mentioned_tickers):
            print (f'(datetime, ticker, topic) values ({index.replace(tzinfo=tz)}, {ticker}, {row.topic})') 
        print ('-'*35)

Headline: BNDES deve começar a venda de ações da carteira pela Vale (VALE3)
(datetime, ticker, topic) values (2020-08-03 21:31:00-03:00, VALE3, Negócios)
(datetime, ticker, topic) values (2020-08-03 21:31:00-03:00, CXSE3, Negócios)
-----------------------------------


---

#### Twitter

1. Get all tweets from the following list of Twitter Users:

- `@Fatosdomercado`
- `@arkoadvice`
- `@BloombergBrasil`
- `@valorinveste`
- `@InvestingBrasil`

2. Extract mentions of listed companies with Rule-Based Method for NER
3. Store into separated `pd.DataFrame` with original tweet data and mentioned ticker

In [153]:
with open('../crawlers/twitter/results/InvestingBrasil.json', encoding='utf8') as json_file:
    data = json.load(json_file) 
    
df_twitter = pd.DataFrame(data)
df_twitter

,tweet_id,created_at,search_dt,text,user_id,screen_name,rt_count,favorite_count
0,1521537839607001088,2022-05-03 14:11:04,2022-05-03 14:15:57,Ações da Alibaba se recuperam em Hong Kong apó...,49292227,InvestingBrasil,1,1
1,1521536893745303552,2022-05-03 14:07:18,2022-05-03 14:15:57,Ações da Pfizer em alta mesmo com previsão de ...,49292227,InvestingBrasil,0,3
2,1521536437241491458,2022-05-03 14:05:30,2022-05-03 14:15:57,Copom inicia reunião para discutir taxa básica...,49292227,InvestingBrasil,1,1
3,1521535957836746753,2022-05-03 14:03:35,2022-05-03 14:15:57,"Estrangeiro tira R$1,7 bi da B3 dia 29 e abril...",49292227,InvestingBrasil,0,3
4,1521535102546853891,2022-05-03 14:00:11,2022-05-03 14:15:57,Solana enfrenta nova instabilidade e fica sete...,49292227,InvestingBrasil,0,4
...,...,...,...,...,...,...,...,...
3242,1497227445119946759,2022-02-25 12:10:14,2022-05-03 14:16:10,Investidor correu para preservar capital nos d...,49292227,InvestingBrasil,0,3
3243,1497220163883647008,2022-02-25 11:41:18,2022-05-03 14:16:10,"#Randon: lucro líquido é de R$ 153,1 mi no 4º ...",49292227,InvestingBrasil,0,4
3244,1497215440854167556,2022-02-25 11:22:32,2022-05-03 14:16:10,"#Marcopolo tem lucro líquido de R$ 65,2 milhõe...",49292227,InvestingBrasil,1,5
3245,1497215330153861122,2022-02-25 11:22:06,2022-05-03 14:16:10,"Petróleo acelera queda e recua 1%, após Rússia...",49292227,InvestingBrasil,2,9


In [154]:
# Convert date to datetime format
df_twitter['date'] = pd.to_datetime(df_twitter['created_at'], format='%Y-%m-%d %H:%M:%S')

# Set date column as index
df_twitter.set_index('date', inplace=True)

# Order by date
df_twitter.sort_index(inplace=True)

In [155]:
df_twitter.sample(5)

,tweet_id,created_at,search_dt,text,user_id,screen_name,rt_count,favorite_count
date,,,,,,,,
2022-03-04 09:42:27,1499726968027295749,2022-03-04 09:42:27,2022-05-03 14:16:09,#Iguatemi compra fatia de 23% no brechó de lux...,49292227,InvestingBrasil,1,8
2022-04-08 11:41:25,1512440483364188174,2022-04-08 11:41:25,2022-05-03 14:16:01,Ouro pode atingir preço recorde com aumento do...,49292227,InvestingBrasil,1,15
2022-04-12 18:23:08,1513991129221509121,2022-04-12 18:23:08,2022-05-03 14:16:01,Maha Energy mira compra de Campo Tartaruga da ...,49292227,InvestingBrasil,0,5
2022-03-11 09:21:50,1502258493662928900,2022-03-11 09:21:50,2022-05-03 14:16:07,Bancos e consultorias agora disputam espaço no...,49292227,InvestingBrasil,0,4
2022-03-08 17:41:56,1501297185903845377,2022-03-08 17:41:56,2022-05-03 14:16:08,Preços em alta abrem espaço para eventual reve...,49292227,InvestingBrasil,0,7


In [156]:
remove_words = ['HTTP', 'HTTPS']

df_twitter_t = pd.DataFrame()

for index, row in df_twitter.iterrows():
    
    title = str(row['text'])

    # Lista de palavras, sem pontuação, com letras maiúsculas
    list_of_words = [re.findall(r"[\w']+|[.,!?;]", x.upper()) for x in title.split()]

    for index, words in enumerate(list_of_words):
        if words and (words[0] in remove_words):
            del list_of_words[index]

    if not list_of_words:
        # Skips blank tweets
        continue 

    # Remover pontuação
    flat_list = [item.translate(str.maketrans('', '', string.punctuation)) for sublist in list_of_words for item in sublist]

    # Verifica ocorrência exata
    mentioned_tickers = set()
    mentioned_tickers = set(flat_list).intersection(tickers)
    
    for title_word in flat_list:
        if title_word in nomes_pregao:
            position = list(ticker_name.values()).index(title_word)
            company_ticker = list(ticker_name.keys())[position]
            mentioned_tickers.add(company_ticker)

    # Store all info if
    # any occurence was identified
    if mentioned_tickers:
        for ticker in list(mentioned_tickers):
            row['ticker'] = ticker
            df_twitter_t = df_twitter_t.append(row, ignore_index=True)

In [157]:
df_twitter_t

,tweet_id,created_at,search_dt,text,user_id,screen_name,rt_count,favorite_count,ticker
0,1497227632240435206,2022-02-25 12:10:59,2022-05-03 14:16:10,#Vale avalia que crise na Ucrânia sustenta pre...,49292227,InvestingBrasil,0.0,6.0,VALE3
1,1497234485460586497,2022-02-25 12:38:13,2022-05-03 14:16:10,Ibovespa tem leve queda após Vale reverter per...,49292227,InvestingBrasil,0.0,5.0,VALE3
2,1497246493404766211,2022-02-25 13:25:56,2022-05-03 14:16:10,Vale: Volume de vendas compensa preços mais ba...,49292227,InvestingBrasil,0.0,3.0,VALE3
3,1497267220782751745,2022-02-25 14:48:17,2022-05-03 14:16:10,Bolsonaro reclama de preço da gasolina e cobra...,49292227,InvestingBrasil,1.0,14.0,PETR4
4,1497305346322411525,2022-02-25 17:19:47,2022-05-03 14:16:10,"#Vale deve pagar R$236,7 mi após descumprir pr...",49292227,InvestingBrasil,0.0,3.0,VALE3
...,...,...,...,...,...,...,...,...,...
183,1520459296189861889,2022-04-30 14:45:19,2022-05-03 14:15:57,"Em 'deserto' de IPOs, B3 retoma movimento com ...",49292227,InvestingBrasil,1.0,9.0,B3SA3
184,1520885734885974018,2022-05-01 18:59:50,2022-05-03 14:15:57,"Depois de 'divórcio', #Itaú volta a ser acioni...",49292227,InvestingBrasil,1.0,18.0,ITUB4
185,1521095289272246275,2022-05-02 08:52:32,2022-05-03 14:15:57,Petrobras inicia produção de FPSO no campo de ...,49292227,InvestingBrasil,0.0,7.0,PETR4
186,1521217600340041728,2022-05-02 16:58:33,2022-05-03 14:15:57,Ações de PetroRio (PRIO3) e Vale (VALE3) são a...,49292227,InvestingBrasil,1.0,9.0,VALE3


In [158]:
df_twitter_t.ticker.value_counts()

PETR4    109
VALE3     59
B3SA3     15
ITUB4      4
EQTL3      1
Name: ticker, dtype: int64